# Building a Search Engine from scratch

We want to build our own search engine from scratch by following this steps:
    
    1. Build our own datasets retrieving almost 20000 html pages referring to a list of ranked animes of all time.
    
    2. Preprocessing of the text
    
    3. Building the search engine to compute synopsis related queries
    
    4. Imporving the search engine computing the cosine similarity

## 0. Useful imports

In [ ]:
import functions as f #import of all the functions we made for this homework.
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import time
import nltk
import csv
from shutil import copyfile
import numpy as np
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')

## 1. Data collection

First we have to retrieve the urls with the BeautifulSoup this is an easy task. We give you the code but we'll provide ypu already with the processed pages it's not mandatory to run it.

In [ ]:
anime = []

for page in tqdm(range(0, 383)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append(link.get('href'))

## Crawler
In this part we'll download the html pages corresponding to the urls. It can be helpful to use a time sleep to avoid being blocked for too much requests. We stored the anime's html in folders dividing them by the correspondent page.

Once we get all the urls in the first 400 pages of the list, we:

1. Download the html corresponding to each of the collected urls.
2. After we collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, we will not lose the data collected up to the stopping point.
3. Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

In [ ]:
!mkdir "./page_folders/"
import os
for page in tqdm(range(1, 384)):
    folder = "page"+str(page)
    path = "./page_folders/"+folder
    os.mkdir(path)

100%|██████████| 192/192 [00:00<00:00, 258.34it/s]


In [ ]:
for page in tqdm(range(0, 383)): 
    folder = "./page_folders/page"+str(page+1)
    update_page = 50*page
    for i in range(0,50):   # 1 -> 50
        url = f'{anime[update_page+i]}'
        response = requests.get(url)   
        filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
        with open(filename,'w', encoding='utf-8') as f:
            f.write(response.text)
        time.sleep(3)

100%|██████████| 9/9 [30:37<00:00, 204.14s/it]


## Parser
Once we have the html we want to collect some informations about the pages. We implemented a parsing function to collect for each anime the following informations:

1. Anime Name (to save as animeTitle): String
2. Anime Type (to save as animeType): String
3. Number of episode (to save as animeNumEpisode): Integer
4. Release and End Dates of anime (to save as releaseDate and endDate): convert both release and end date into datetime format.
5. Number of members (to save as animeNumMembers): Integer
6. Score (to save as animeScore): Float
7. Users (to save as animeUsers): Integer
8. Rank (to save as animeRank): Integer
9. Popularity (to save as animePopularity): Integer
10. Synopsis (to save as animeDescription): String
11. Related Anime (to save as animeRelated): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. List of strings.
12. Characters (to save as animeCharacters): List of strings.
13. Voices (to save as animeVoices): List of strings
14. Staff (to save as animeStaff): Include the staff name and their responsibility/task in a list of lists.

For each anime, we create an ***anime_i.tsv*** file of this structure:
- animeTitle \t animeType \t  ... \t animeStaff 


If an information is missing, we just leave it as an empty string. Example:

- animeTitle \t animeType \t  ... \t animeStaff
    
- Fullmetal Alchemist: Brotherhood \t TV \t ... \t [['Cook, Justin', 'Producer'], ['Irie, Yasuhiro', 'Director, Episode Director, Storyboard'], ['Yonai, Noritomo', 'Producer'], ['Mima, Masafumi', 'Sound Director']]



We create a folder to store all the tsv's

In [ ]:
!mkdir "./tsv_files/"
import os
for page in tqdm(range(1, 384)):
    folder = "page"+str(page)
    path = "./tsv_files/"+folder
    os.mkdir(path)

Then we parse the html and convert them into tsv's

In [ ]:
for j in tqdm(range(0,383)): 
  tqdm.write(f'   page{j+1}')
  update_page = 50*j
  for i in range(0,50):
      f.html_parsing(f'tsv_files/page{j+1}/anime_{update_page+i+1}.tsv', 
                   f'./page_folders/page{j+1}/anime_{update_page+i+1}.html')

In [ ]:
with open('./tsv_files/header.tsv', 'wt') as out_file:
    anime_tsv = csv.writer(out_file, delimiter='\t')
    anime_tsv.writerow(['animeTitle', 'animeType', 'animeNumEpisode', 'animeRelDate', 'animeEndDate', 
                        'animeMembers', 'animeScore', 'animeUsers', 'animeRank', 'animePopularity', 'animeSynopsis',
                        'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff'])

Once we collected all the tsv's file we have to clean them (replacing empty lists w/ empty strings) 

In [ ]:
for j in tqdm(range(0,383)): 
  tqdm.write(f'   page{j+1}')
  update_page = 50*j
  for i in range(0,50):
        f = open(f'./tsv_files/page{j+1}/anime_{update_page+i+1}.tsv', mode = 'r', encoding='utf-8')
        txt  = f.read().strip()
        f.close()
        f = open(f'./tsv_files/page{j+1}/anime_{update_page+i+1}.tsv', mode = 'w', encoding='utf-8')
        f.write(txt.replace('\\', ''))
        f.close()

Last step is to merge all the tsv in a single file called _tsv_merged_

In [ ]:
for j in tqdm(range(0,383)): 
  tqdm.write(f'   page{j+1}')
  update_page = 50*j
  for i in range(0,50):
    copyfile(f'./tsv_files/page{j+1}/anime_{update_page+i+1}.tsv', 
                   f'./tsv_merged/anime_{update_page+i+1}.tsv')

In [ ]:
with open(f"./tsv_files/header.tsv", encoding='utf-8') as f:
    header = f.read().strip().split('\t')
    f.close()

## 2. Building the Search Engine

The fundametal steps to build a search engine for documents are:

1. Pre-processing of the documents ending up with a dictionary of the document id as key and the list of tokens of the document as values.

2. Building the inverted index such thta for each token we have all the documents in which is contained. In the second part we'll consider as value a tuple of document and tfidf of the token for that document.

3. Implementing a function thta given a query retrieves the related documents. 

The first step is to build a dictionary of dictionaries, for every anime we'll have a dictionary containing all the informations retrieved earlier. The query will be focused only on the synopsys for now.

In [ ]:
D = {} # this dictionary will store all the dictionaries of the single animes
for i in tqdm(range(0,19126)):

    d = f.Dict(f"/content/drive/MyDrive/ADM-HW3/tsv_merged/anime_{i+1}.tsv")
    D[f"anime_{i+1}"] = d

### Synopsis preprocessing
First, we must pre-process all the information collected for each anime by:

- Removing stopwords
- Removing punctuation
- Stemming
- Anything else we think it's needed

For this purpose, we use the nltk library.

Once we preprocessed with the _text_preprocessing_ function we collect all the unique tokens

In [ ]:
# first we preprocess the synopsis by lowering the case removing stopwords, punctuation and numbers and finally stemming
all_tokens = []

for key in tqdm(D):

    out, tkns = f.text_preprocessing(D[key]['animeSynopsis'], lower = True, numbers = True, stemming = True)

    for token in tkns:
        if token not in all_tokens: # we want to store all the unique tokens in a list
          all_tokens.append(token)

# save all tokens in a txt file
f = open(f"/content/drive/MyDrive/ADM-HW3/tokens.txt", "w", encoding='utf-8')
for i in all_tokens:
    f.write(i + '\n')
f.close()

100%|██████████| 19125/19125 [01:46<00:00, 179.78it/s]


We map each token to an integer in the following dictionary, this we'll be useful later

In [ ]:
# we map every unique token to an integer and store it in a dictionary
term_id = range(1, len(all_tokens))
vocab = {k: v for k,v in zip(all_tokens, term_id)}

Then we store in two arrays the tokens and the synopsis and we'll use them as input in the _inv_index_ function that will build the inverted index

In [ ]:
# we store the tokens and the synopsis in two numpy arrays

tok = list(f.vocab.keys())
tok = np.array(tok)

syn = []
for key in tqdm(D):

    a, b = f.text_preprocessing(D[key]['animeSynopsis'], lower = True, numbers = True, stemming = True)
    if a == 'synopsi inform ad titl help improv databas ad synopsi': 
      syn.append([''])
      f.preproc_D[key]['animeSynopsis'] = [' ']

    else: 
      
      f.preproc_D[key]['animeSynopsis'] = b


syn = np.array(syn)

In [ ]:
# we compute the inverted index

inv_index = f.inverted_index(tok, syn)

100%|██████████| 43615/43615 [18:09<00:00, 40.04it/s]


### Executing the queries 
Given a query, that we let the user enter:

*saiyan race*

the Search Engine is supposed to return a list of documents.

**What documents do we want?**

Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

- **animeTitle**
- **animeDescription**
- **Url**


In [2]:
f.search()

search bar:  saiyan race


,animeTitle,animeDescription,Url
1,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n
2,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
3,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...
4,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...


## Cosine similarity and Tf-idf

At this point we would like to have a more powerful search engine that given a query will compute the cosine similarity and return the best ranked results by using a MinHeap

Our second Inverted Index is of this format:

{

term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],

term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],

...}

Practically, for each word we want the list of documents in which it is contained in, and the relative *tfIdf* score.

**Tip**: to compute the tfidf you can also use the sci-kit library

In [21]:
# this time we store the synopsis in a dicitionary

anime_id, syn = [], []
for key in tqdm(f.D):
    anime_id.append(key)
    a, b = f.text_preprocessing(f.D[key]['animeSynopsis'], lower = True, numbers = True, stemming = True)
    syn.append(b)

synopsis = {k:v for k,v in zip(anime_id, syn)}

100%|██████████| 19125/19125 [00:41<00:00, 463.16it/s]


Then we compute the tf-idf as follows:

It is the combination of *Term Frequency (TF)* and *Inverse Data Frequency (IDF)*.

TF is the number of times a word *t* appears in a document *d* divided by the total number of words in the document. Every document has its own term frequency:

$tf_{t,d}=\frac{n_{t,d}}{\sum_{t'\in d} n_{t',d}}$



The IDF is a measure of how much information the word provides, i.e. if it's common or rare across all documents.
IDF is the log of the number of all documents *N* divided by the number of documents *d* that contain the word *t*. IDF determines the weight of rare words across all documents in the corpus:

$idf(t,D)=\log \left(\frac{N}{| \{ d\in D: t\in D \} | }\right)$

TF-IDF is given by the multiplication of TF and IDF:

$w_{t,d,D}=tf_{t,d} \times idf(t,D)$

In [ ]:
tok = list(f.vocab.keys())
tok = np.array(tok)

for j in tqdm(range(0, len(tok))):
    term_j,  k = tok[j], 0

    for i in f.inv_index[term_j]:
        try:
            doc_i = synopsis[i]
            tfidf = ( doc_i.count(term_j) / len(doc_i) ) * ( np.log10( len(synopsis) / len(f.inv_index[term_j]) ))
            f.inv_index[term_j][k]  = (i, tfidf)
            k += 1
        except KeyError: pass
        continue

100%|██████████| 43615/43615 [00:04<00:00, 10526.90it/s]


### Execute the query

In [13]:
f.search_cosine()

search bar: alchemist


,animeTitle,animeDescription,Url,Similarity
1,Shinmai Renkinjutsushi no Tenpo Keiei,Shoot for the stars! I'm going to be the count...,https://myanimelist.net/anime/49849/Shinmai_Re...,0.76
2,Arcana Famiglia: Capriccio - stile Arcana Fami...,"After toiling away in his lab, the alchemist J...",https://myanimelist.net/anime/15411/Arcana_Fam...,0.14
3,Ulysses: Jehanne Darc to Renkin no Kishi,"The story is set in the 15th century, during t...",https://myanimelist.net/anime/36510/Ulysses__J...,0.13


## Conclusions

This is a Naive search engine that could be imporved for example by using hash functions to do the queries or just changing the way a query is computed adding filters for example or just defining a more discriminative score.